# Hello hello hello
### thank you for joining us today

#### List of dictionaries:

##### Day0: 3 dictionaries total 
(1. Time cell in task 2. Place cell in task 3. Time cell in reward)

time_binned_dict_day0_reward<br>
space_binned_dict_day0<br>
time_binned_dict_day0<br>


##### Day4: 4 dictionaries total 
(1. Time cell in task (1 dictionary per odor) 2. Place cell in task )

time_binned_dict_day4odor1<br>
time_binned_dict_day4odor2<br>
space_binned_dict_day4odor1<br>
space_binned_dict_day4odor2<br>

    
##### Day 5: 2 dictionaries total 
(1. Time cell in task (only no odor) 2. Place cell in task (only no odor)

time_binned_dict_day5<br>
space_binned_dict_day5<br>




In [84]:
import pandas as pd
df = pd.read_csv('wfC318/Day_0/Raw_files_csv/combined_behavior_and_s.csv', header=None)

#adding labels to the data
labs = ['frame', 'time', 'odor_valve', 'total_licks', 'total_rewards', 'trial', 
        'valve_open', 'virt_dist', 'total_distance', 'lap', 'trial_type', 'speed']

#name the rest of the columns with neuron index
for i in range(0,255):
    labs.append(str(i))
    
df.columns = labs
print(df.head())

   frame  time  odor_valve  total_licks  total_rewards  trial  valve_open  \
0      1    34           0            0              0      0           0   
1      2    84           0            0              0      0           0   
2      3   133           0            0              0      0           0   
3      4   183           0            0              0      0           0   
4      5   233           0            0              0      0           0   

   virt_dist  total_distance  lap  ...  245     246  247  248  249  250  251  \
0         13               0    0  ...  0.0  0.0000  0.0  0.0  0.0  0.0  0.0   
1         26               0    0  ...  0.0  2.7654  0.0  0.0  0.0  0.0  0.0   
2         36               0    0  ...  0.0  0.0000  0.0  0.0  0.0  0.0  0.0   
3         50               0    0  ...  0.0  0.0000  0.0  0.0  0.0  0.0  0.0   
4         65               0    0  ...  0.0  0.0000  0.0  0.0  0.0  0.0  0.0   

   252  253  254  
0  0.0  0.0  0.0  
1  0.0  0.0  0.0  

In [85]:
#remove all trials after 30
df = df[df["trial"] < 31]

#restart time so it increments within a given trial only
#do not run this cell more than once or it will fuck everything up
trial_prev, time_prev, end_time = 0,0,0
for i, j in df.iterrows(): 
    trial = int(j['trial'])
    time = int(j['time'])
    if trial != trial_prev:
        end_time = time_prev
        #print("\n")
    df.at[i,'time'] = time - end_time
    trial_prev = trial
    time_prev = time

In [86]:
#remove all cells where distance exceeds 4,000cm
df = df[df["virt_dist"] < 4000]

## The following bins the data
changeTime() resets the time associated with each imaging frame so the clock resets after each trial completes. 

binning() bins the data into an outter dictionary where each (key,value) corresponds to each trial. Each value within this dictionary is another dictionary where each key is a 200ms time interval and the values are 4 imaging frames from that particular trial that fall within that time range (for example, bin "200" houses data that occurs between 0 and 200ms). 



### Full binning from 0 to 4000cm


In [80]:
import pandas as pd
import numpy as np
import time as t

# Things to set:
#filename = 'wfC318/Day_4/raw_files_csv/combined_behavior_and_s_COPY.csv'
filename = 'wfC318/Day_4/raw_files_csv/combined_behavior_and_s.csv'
#filename = 'wfC318/Day_0/Raw_files_csv/combined_behavior_and_s.csv'

# How many trials we are looking at (ex: 31 for day 0, 34 for day 4)
trial_count = 31


data = pd.read_csv(filename, header=None)

# Will change neuron firing data to (0,1) binary indicators
data_for_freq = pd.read_csv(filename, header=None)

def changeTime(data_name):
    trial_prev, time_prev, end_time = 0,0,0
    for i, j in data_name.iterrows():
        trial = int(j[9])
        time = int(j[1])
        if trial != trial_prev:
            end_time = time_prev
        data_name.at[i,1] = time - end_time
        trial_prev = trial
        time_prev = time
    print("time changed")
    
changeTime(data)
changeTime(data_for_freq)


length = len(data)
data = data[data[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm
data_for_freq = data_for_freq[data_for_freq[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm


time = data[1]
odor = data[10]
distance = data[7]
lap = data[9]
# neuron data starts at column "13", indexed from 0 so the first column of neuron data is data[12]
cell = data.loc[:, 12:(len(data.columns)-1)]  # neuron data

#trial_count = max(lap)  # all of them


# using lambda function instead
def binaryIndicator(x):
    if x > 0:
        x = 1
    else:
        x = 0
    return x


# Using second copy of data to change neuron data to (0,1) to calculate 10% threshold
# Changing neuron firing data to 0,1 binary
num_of_cols = len(data_for_freq.columns)
for j in range(12, num_of_cols):  # for all neuron columns
    # setting value to 1 if val>0, 0 otherwise
    data_for_freq[j] = data_for_freq[j].apply(lambda x: 1 if (x > 0) else 0)
    #data_for_freq[j] = data_for_freq[j].apply(binaryIndicator)


lap_dictionary = {}
for i in range(0, trial_count):  # for every lap
    lap_dictionary[i] = data.loc[data[9] == i]  # associate all lap data with that lap


def findShortestTrial():
    min_time = 999999999
    max_time = 0
    shortest_trial = lap_dictionary[0]
    longest_trial = lap_dictionary[0]
    time_list = []
    for trial in lap_dictionary:
        first = lap_dictionary[trial].iloc[0]
        last = lap_dictionary[trial].iloc[-1]
        time = last[1] - first[1]
        time_list.append(time)
        if(time < min_time):
            min_time=time
            shortest_trial = trial
        if (time > max_time):
            max_time = time
            longest_trial = trial
    return shortest_trial, min_time, longest_trial, max_time

shortest_trial, min_time,longest_trial, max_time = findShortestTrial()
print("Shortest Trial:", shortest_trial, "Time: ", min_time)
print("Longest Trial:", longest_trial, "Time: ", max_time)


# Determining number of bins
num_of_time_bins = int(min_time // 200)
num_of_space_bins = 4000//100 # 100cm bin width 
print("number of time bins:", num_of_time_bins)


# create empty row to store neuron data frequencies
empty_row = {}
for k in range(0, num_of_cols-13):  # 255 or 459 columns for example
    empty_row[k] = 0.0

def findMeanValues():
    flagCounter = 0
    test_dictionary_for_laps = {}
    start = t.time()  # for reference
    for i in lap_dictionary:  # for every trial
        # average columns in that trial to test (eventually, in bin instead)
        current_row = pd.DataFrame(empty_row, index=[0])  # create pandas dataframe from a row of 0's (reset every lap)
        for j in range(12, num_of_cols):  # for all neuron columns
            #current_row[j - 12] = data[j].mean()
            current_row[j-12] = lap_dictionary[i][j].mean()  # fill each column spot in that row with the mean of all values in that column
            # note: because we set values to (0,1), this mean becomes frequency relative to that trial
        test_dictionary_for_laps[i] = current_row  # set dictionary item equal to that row
    end = t.time()
    return test_dictionary_for_laps

test_dictionary_for_laps = findMeanValues()

NEW_lap_dictionary = {}
for i in range(0, trial_count):  # for every lap
    NEW_lap_dictionary[i] = data.loc[data[9] == i]  # associate all lap data with that lap
    #print(NEW_lap_dictionary[i].iloc[:,:13], "\n\n")
    
def timeBinning():
    # each key value pair will be (Trial_number, [ [Bin1], [Bin2], [Bin3], ..., [Bin43] ])
    dict_of_all_bins = {}
    for i in NEW_lap_dictionary:  # for every trial
        list_of_bins = {}  # each (key,value) pair will be start-time of bin and list of points in bin for each trial
        # indexed from 1
        for bin_number in range(1, num_of_time_bins+1):
            bin_ending_time = bin_number*200
            last_time = (bin_number-1)*200
            # add all data points from the current trial in lap_dictionary to bin if time is in bin range
            data_points = NEW_lap_dictionary[i].loc[(NEW_lap_dictionary[i][1] < bin_ending_time) & (NEW_lap_dictionary[i][1] >= last_time) ]
            list_of_bins[bin_ending_time] = data_points
        dict_of_all_bins[i] = list_of_bins

    return dict_of_all_bins
time_binned_dic_day0 = timeBinning()
print("temporally binned dictionary")

def spaceBinning():
     # each key value pair will be (Trial_number, [ [Bin1], [Bin2], [Bin3], ..., [Bin43] ])
    dict_of_all_bins = {}
    for i in NEW_lap_dictionary:  # for every trial
        #print("TEST", NEW_lap_dictionary[i], "\n\n")
        list_of_bins = {}  # each (key,value) pair will be start-time of bin and list of points in bin for each trial
        # indexed from 1
        for bin_number in range(1, num_of_space_bins+1):
            bin_starting_dist = bin_number*100  # 10 cm bin width
            last_dist = (bin_number-1)*100
            # add all data points from the current trial in lap_dictionary to bin if time is in bin range
            data_points = NEW_lap_dictionary[i].loc[(NEW_lap_dictionary[i][7] < bin_starting_dist) & (NEW_lap_dictionary[i][7] >= last_dist) ]
            list_of_bins[bin_starting_dist] = data_points
            #print(data_points)
        dict_of_all_bins[i] = list_of_bins

    return dict_of_all_bins
space_binned_dict_day0 = spaceBinning()
print("spatially binned dictionary")




time changed
time changed
Shortest Trial: 4 Time:  6901.0
Longest Trial: 30 Time:  12502.0
number of time bins: 34
temporally binned dictionary
spatially binned dictionary


## Getting neurons as column selections that fire in >10% of all trials

In [81]:
"""
for trial, bins in binned_dic.items(): 
    for time, activities in bins.items():
        for cell, values in activites.items():
            if cell<=12: # skipping over behavior & metadata 
                continue
            
""" 

lap_dict_for_freq = {}
for i in range(0, trial_count):  # for every lap
    lap_dict_for_freq[i] = data_for_freq.loc[data_for_freq[5] == i]  # associate all lap data with that lap


# create empty row to store neuron data frequencies
empty_row = {}
for k in range(0, num_of_cols-13):  # 255 or 459 columns for example
    empty_row[k] = 0.0

def findFrequencies():
    flagCounter = 0
    test_dictionary_for_laps = {}
    #start = t.time()
    for i in lap_dict_for_freq:  # for every trial
        # average columns in that trial to test
        current_row = pd.DataFrame(empty_row, index=[0])  # create pandas dataframe from a row of 0's (reset every lap)
        for j in range(12, num_of_cols):  # for all neuron columns
            current_row[j-12] = lap_dict_for_freq[i][j].mean()  # fill each column spot in that row with the mean of all values in that column

            # note: because we set values to (0,1), this mean becomes frequency relative to that trial
        test_dictionary_for_laps[i] = current_row  # set dictionary item equal to that row
        """
        while(flagCounter<5):
            print(type(current_row))
            flagCounter+=1
        """
        
    print(type(test_dictionary_for_laps))
    #end = t.time()
    #print("Time:", end-start)
    print("freq found")
    return test_dictionary_for_laps

frequencies  = findFrequencies()


<class 'dict'>
freq found


## Neuron Selection

In [88]:
def neuronSelection():
    freq = {}
    selection = {}
    #for k in range(0, num_of_cols-13):  # 255 or 459 columns for example
    #for k in range(0, num_of_cols-12):  # 255 or 459 columns for example
    for k in range(12, num_of_cols):  # 255 or 459 columns for example

        freq[k] = 0
        selection[k] = 0
    
    trial_count = len(frequencies)
    
    ### Set threshold here!
    threshold = trial_count // 10 # 10% of trials
    print("Trials:",trial_count, "Threshold:", threshold)
    for trial in frequencies:     
        for i in range(0, num_of_cols-12): # 0,254            
            if frequencies[trial].iloc[0,i]>0:
                freq[i+12]+=1 
    
    #print(freq)
    index2 = 12 #0    
    counter=0
    for val in freq:
        if freq[val]<threshold:
            counter+=1
            selection[index2]=0
        else:
            selection[index2]=1
        index2+=1
    
    print("CUTTING:",counter," neurons \nKEEPING:",len(freq)-counter)
    return freq, selection

freq, selection = neuronSelection()
#print(selection)

Trials: 31 Threshold: 3
CUTTING: 0  neurons 
KEEPING: 459


## Getting neurons as column selections that fire in >10% of all trials in same half of trial

In [90]:
first_lap_dict_for_freq = {}
second_lap_dict_for_freq = {}

for i in range(0, trial_count):  # for every lap
    #two_halves_of_trial.append([data_for_freq.loc[data_for_freq[9] == i & data_for_freq[8] < 2000]]) # first half
    #two_halves_of_trial.append([data_for_freq.loc[data_for_freq[9] == i & data_for_freq[8] >= 2000]]) # second half
    #lap_dict_for_freq[i] = two_halves_of_trial  # associate all lap data with that lap
    first_lap_dict_for_freq[i] = data_for_freq.loc[(data_for_freq[5] == i) & (data_for_freq[7] < 2000)]
    second_lap_dict_for_freq[i] = data_for_freq.loc[(data_for_freq[5] == i) & (data_for_freq[7] >= 2000)]

#print(first_lap_dict_for_freq)
#print(second_lap_dict_for_freq)


# create empty row to store neuron data frequencies
empty_row = {}
for k in range(0, num_of_cols-13):  # 255 or 459 columns for example
    empty_row[k] = 0.0

def findFrequencies(dict_for_laps):
    flagCounter = 0
    test_dictionary_for_laps = {}
    for i in dict_for_laps:  # for every trial
        # average columns in that trial to test
        current_row = pd.DataFrame(empty_row, index=[0])  # create pandas dataframe from a row of 0's (reset every lap)
        for j in range(12, num_of_cols):  # for all neuron columns
            #current_row[j-12] = dict_for_laps[i][j].mean()  # fill each column spot in that row with the mean of all values in that column
            current_row[j-12] = dict_for_laps[i][j].sum()  # fill each column spot in that row with the sum of all values in that column
        test_dictionary_for_laps[i] = current_row  # set dictionary item equal to that row
    print("freq found")
    return test_dictionary_for_laps

frequencies1  = findFrequencies(first_lap_dict_for_freq)
frequencies2  = findFrequencies(second_lap_dict_for_freq)

#print(len(frequencies1))
#print(len(frequencies2))


freq found
freq found


## Neuron Frequency (two halves version)

In [91]:
def neuronSelection(frequencies):
    freq = {}
    selection = {}
    #for k in range(0, num_of_cols-13):  # 255 or 459 columns for example
    #for k in range(0, num_of_cols-12):  # 255 or 459 columns for example
    for k in range(12, num_of_cols):  # 255 or 459 columns for example
        freq[k] = 0
        selection[k] = 0
    
    trial_count = len(frequencies)
    
    ### Set threshold here!
    threshold = trial_count // 10 # 10% of trials
    print("Trials:",trial_count, "Threshold:", threshold)
    for trial in frequencies:     
        for i in range(0, num_of_cols-12): # 0,254            
            if frequencies[trial].iloc[0,i]>0:
                freq[i+12]+=1 
    
    #print(freq)
    index2 = 12 #0    
    counter=0
    for val in freq:
        if freq[val]<threshold:
            counter+=1
            selection[index2]=0
        else:
            selection[index2]=1
        index2+=1
    
    print("CUTTING:",counter," neurons \nKEEPING:",len(freq)-counter)
    return freq, selection

freq1, selection1 = neuronSelection(frequencies1)
#print(selection1)

freq2, selection2 = neuronSelection(frequencies2)
#print(selection2)


def overlap(selection1, selection2):
    # create empty row to store neuron data frequencies
    row = {}
    counter = 0
    mismatch_counter = 0
    for k in range(12, num_of_cols):  # 255 or 459 columns for example
        #print(selection1[k], selection2[k])
        if (selection1[k] == 1) and (selection2[k] == 1):
            row[k] = 1
        elif selection1[k] != selection2[k]:
            row[k] = 0
            mismatch_counter+=1
        else: 
            row[k] = 0
            counter+=1
    print("\nCUTTING:",counter," neurons \nKEEPING:",len(freq)-(counter+mismatch_counter), "\nmismatches:", mismatch_counter)
    return row

freqs = overlap(selection1, selection2)
#print(freqs)

        

Trials: 31 Threshold: 3
CUTTING: 25  neurons 
KEEPING: 434
Trials: 31 Threshold: 3
CUTTING: 15  neurons 
KEEPING: 444

CUTTING: 1  neurons 
KEEPING: 420 
mismatches: 38


### Binning reward period (4000cm+ )

In [84]:
import pandas as pd
import numpy as np
import time as t

#filename = 'wfC318/Day_4/raw_files_csv/combined_behavior_and_s.csv'
filename = 'wfC318/Day_0/Raw_files_csv/combined_behavior_and_s.csv'

# How many trials we are looking at (ex: 31 for day 0, 34 for day 4)
trial_count = 31

data = pd.read_csv(filename, header=None)

def changeTime(data_name):
    trial_prev, time_prev, end_time = 0,0,0
    for i, j in data_name.iterrows():
        trial = int(j[9])
        time = int(j[1])
        if trial != trial_prev:
            end_time = time_prev
        data_name.at[i,1] = time - end_time
        trial_prev = trial
        time_prev = time
    print("time changed")
    
changeTime(data)

length = len(data)
#data = data[data[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm

time = data[1]
odor = data[10]
distance = data[7]
lap = data[9]
# neuron data starts at column "13", indexed from 0 so the first column of neuron data is data[12]
cell = data.loc[:, 12:(len(data.columns)-1)]  # neuron data

#trial_count = max(lap)  # all of them

lap_dictionary = {}
for i in range(0, trial_count):  # for every lap
    lap_dictionary[i] = data.loc[data[9] == i]  # associate all lap data with that lap

def findShortestTrial():
    min_time = 999999999
    max_time = 0
    shortest_trial = lap_dictionary[0]
    longest_trial = lap_dictionary[0]
    time_list = []
    for trial in lap_dictionary:
        # first row where mouse passes 4000cm, indicated by lapCount increment
        first = lap_dictionary[trial].loc[ lap_dictionary[trial][5] > trial ]
        #print(first)
        last = lap_dictionary[trial].iloc[-1]
        time = last[1] - first.iloc[0,1]
        #print(last[1], first.iloc[0,1], time)
        time_list.append(time)
        if(time < min_time):
            min_time = time
            shortest_trial = trial
        if (time > max_time):
            max_time = time
            longest_trial = trial
    return shortest_trial, min_time, longest_trial, max_time

shortest_trial, min_time,longest_trial, max_time = findShortestTrial()
print("Shortest Trial:", shortest_trial, "Time: ", min_time)
print("Longest Trial:", longest_trial, "Time: ", max_time)


# Determining number of bins
num_of_time_bins = int(min_time // 200)
num_of_space_bins = 4000//100 # 100cm bin width 
print("number of time bins:", num_of_time_bins)

NEW_lap_dictionary = {}
for i in range(0, trial_count):  # for every lap
    NEW_lap_dictionary[i] = data.loc[(data[9] == i) & (data[7] > 4000)]  # associate all lap data with that lap
    #print(NEW_lap_dictionary[i])

def timeBinning():
    # each key value pair will be (Trial_number, [ [Bin1], [Bin2], [Bin3], ..., [Bin43] ])
    dict_of_all_bins = {}
    for i in NEW_lap_dictionary:  # for every trial
        #print("TEST", NEW_lap_dictionary[i], "\n\n")
        list_of_bins = {}  # each (key,value) pair will be start-time of bin and list of points in bin for each trial
        # indexed from 1
        first = NEW_lap_dictionary[i].loc[ NEW_lap_dictionary[i][5] > i ]
        #print("HEREEE", first)
        inceptionTIME = first.iloc[0,1]
        
        #inceptionTIME = 
        for bin_number in range(1, num_of_time_bins+1):
            bin_ending_time = bin_number*200
            last_time = (bin_number-1)*200
            # add all data points from the current trial in lap_dictionary to bin if time is in bin range
            data_points = NEW_lap_dictionary[i].loc[(NEW_lap_dictionary[i][1] < bin_ending_time + inceptionTIME) & (NEW_lap_dictionary[i][1] >= last_time+inceptionTIME) ]
            list_of_bins[bin_ending_time] = data_points
            #print(data_points)
        dict_of_all_bins[i] = list_of_bins

    return dict_of_all_bins
time_binned_dict_day0_reward = timeBinning()
print("temporally binned dictionary")

#print(binned_dic)


time changed
Shortest Trial: 20 Time:  4250.0
Longest Trial: 9 Time:  4652.0
number of time bins: 21
temporally binned dictionary


<br>
<br>
<br>



# DAY 4 using different odor trials without reward period


In [85]:
import pandas as pd
import numpy as np
import time as t

filename = 'wfC318/Day_4/raw_files_csv/combined_behavior_and_s.csv'
#filename = 'wfC318/Day_0/Raw_files_csv/combined_behavior_and_s.csv'

# How many trials we are looking at (ex: 31 for day 0, 34 for day 4)
trial_count = 31

data_day4_tot = pd.read_csv(filename, header=None)
data_day4_odor1 = pd.read_csv(filename, header=None)
data_day4_odor2 = pd.read_csv(filename, header=None)

def changeTime(data_name):
    trial_prev, time_prev, end_time = 0,0,0
    for i, j in data_name.iterrows():
        trial = int(j[9])
        time = int(j[1])
        if trial != trial_prev:
            end_time = time_prev
        data_name.at[i,1] = time - end_time
        trial_prev = trial
        time_prev = time
    print("time changed")

changeTime(data_day4_tot)
changeTime(data_day4_odor1)
changeTime(data_day4_odor2)


data_day4_tot = data_day4_tot[data_day4_tot[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm

data_day4_odor1 = data_day4_odor1[data_day4_odor1[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm
data_day4_odor1 = data_day4_odor1[data_day4_odor1[10] == 1]

data_day4_odor2 = data_day4_odor2[data_day4_odor2[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm
data_day4_odor2 = data_day4_odor2[data_day4_odor2[10] == 2]


unique_trial_ids1 = data_day4_odor1[9].unique()
unique_trial_ids2 = data_day4_odor2[9].unique()


#trial_count = max(lap)  # all of them

"""
Dictionary of all trials
"""
lap_dict_tot = {}
for i in range(0, trial_count):  # for every lap
    lap_dict_tot[i] = data_day4_tot.loc[data_day4_tot[9] == i]  # associate all lap data with that lap



"""
Dictionary of just day 4 odor 1
"""
trial_indices1 = []
lap_dict_day4odor1 = {}
counter=0
for i in range(0, len(unique_trial_ids1)):  # for every lap
    idx = int(unique_trial_ids1[counter])
    trial_indices1.append(idx)
    # associate all lap data with that lap, when there is no odor, 
    # so we can no longer directly associate the dictionary index with the trial number
    lap_dict_day4odor1[idx] = data_day4_odor1.loc[data_day4_odor1[9] == idx]
    counter+=1

    

"""
Dictionary of just day 4 odor 2
"""   
trial_indices2 = []
lap_dict_day4odor2 = {}
counter=0
for i in range(0, len(unique_trial_ids2)):  # for every lap
    idx = int(unique_trial_ids2[counter])
    trial_indices2.append(idx)
    # associate all lap data with that lap, when there is no odor, 
    # so we can no longer directly associate the dictionary index with the trial number
    lap_dict_day4odor2[idx] = data_day4_odor2.loc[data_day4_odor2[9] == idx]
    counter+=1



def findShortestTrial(lap_dictionary):
    min_time = 999999999
    max_time = 0
    shortest_trial = lap_dictionary[0]
    longest_trial = lap_dictionary[0]
    time_list = []
    for trial in lap_dictionary:
        first = lap_dictionary[trial].iloc[0]
        last = lap_dictionary[trial].iloc[-1]
        time = last[1] - first[1]
        time_list.append(time)
        if(time < min_time):
            min_time=time
            shortest_trial = trial
        if (time > max_time):
            max_time = time
            longest_trial = trial
    #print("AVG trial duration:", sum(time_list)/len(time_list))
    return shortest_trial, min_time, longest_trial, max_time

shortest_trial, min_time,longest_trial, max_time = findShortestTrial(lap_dict_tot)
print("Shortest Trial:", shortest_trial, "Time: ", min_time)
print("Longest Trial:", longest_trial, "Time: ", max_time)

# Determining number of bins
num_of_time_bins = int(min_time // 200)
num_of_space_bins = 4000//100 # 100cm bin width 
print("number of time bins:", num_of_time_bins)


def timeBinning(lap_dictionary):
    # each key value pair will be (Trial_number, [ [Bin1], [Bin2], [Bin3], ..., [Bin43] ])
    dict_of_all_bins = {}
    for i in lap_dictionary:  # for every trial
        list_of_bins = {}  # each (key,value) pair will be start-time of bin and list of points in bin for each trial
        # indexed from 1
        for bin_number in range(1, num_of_time_bins+1):
            bin_ending_time = bin_number*200
            last_time = (bin_number-1)*200
            # add all data points from the current trial in lap_dictionary to bin if time is in bin range
            data_points = lap_dictionary[i].loc[(lap_dictionary[i][1] < bin_ending_time) & (lap_dictionary[i][1] >= last_time) ]
            list_of_bins[bin_ending_time] = data_points
        dict_of_all_bins[i] = list_of_bins
    return dict_of_all_bins


def spaceBinning(lap_dictionary):
     # each key value pair will be (Trial_number, [ [Bin1], [Bin2], [Bin3], ..., [Bin43] ])
    dict_of_all_bins = {}
    for i in lap_dictionary:  # for every trial
        list_of_bins = {}  # each (key,value) pair will be start-time of bin and list of points in bin for each trial
        # indexed from 1
        for bin_number in range(1, num_of_space_bins+1):
            bin_starting_dist = bin_number*100  # 10 cm bin width
            last_dist = (bin_number-1)*100
            # add all data points from the current trial in lap_dictionary to bin if time is in bin range
            data_points = lap_dictionary[i].loc[(lap_dictionary[i][7] < bin_starting_dist) & (lap_dictionary[i][7] >= last_dist) ]
            list_of_bins[bin_starting_dist] = data_points
        dict_of_all_bins[i] = list_of_bins
    return dict_of_all_bins



time_binned_dict_day4odor1 = timeBinning(lap_dict_day4odor1)
time_binned_dict_day4odor2 = timeBinning(lap_dict_day4odor2)
print("temporally binned dictionary")

space_binned_dict_day4odor1 = spaceBinning(lap_dict_day4odor1)
space_binned_dict_day4odor2 = spaceBinning(lap_dict_day4odor2)
print("spatially binned dictionary")



time changed
time changed
time changed
Shortest Trial: 4 Time:  6901.0
Longest Trial: 30 Time:  12502.0
number of time bins: 34
temporally binned dictionary
spatially binned dictionary


<br>
<br>
<br>
<br>
<br>
<br>


# DAY 5 using non-odor trials 
### ignoring trials in which an odor is released

In [86]:
import pandas as pd
import numpy as np
import time as t

filename = 'wfC318/Day_5/combined_behavior_and_s_day5.csv'
# How many trials we are looking at (ex: 31 for day 0, 34 for day 4)
trial_count = 31

data = pd.read_csv(filename, header=None)
#data_for_freq = pd.read_csv(filename, header=None)

def changeTime(data_name):
    trial_prev, time_prev, end_time = 0,0,0
    for i, j in data_name.iterrows():
        trial = int(j[9])
        time = int(j[1])
        if trial != trial_prev:
            end_time = time_prev
        data_name.at[i,1] = time - end_time
        trial_prev = trial
        time_prev = time
    print("time changed")
    
changeTime(data)

length = len(data)
data = data[data[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm
data = data[data[10] <= 1]  # removes all trials where trial type=2(there's odor)

unique_trial_ids = data[9].unique()
#print(type(unique_trial_ids), type(unique_trial_ids[0]))

time = data[1]
odor = data[10]
distance = data[7]
lap = data[9]
# neuron data starts at column "13", indexed from 0 so the first column of neuron data is data[12]
cell = data.loc[:, 12:(len(data.columns)-1)]  # neuron data

#trial_count = max(lap)  # all of them


# using lambda function instead
def binaryIndicator(x):
    if x > 0:
        x = 1
    else:
        x = 0
    return x

"""
# Using second copy of data to change neuron data to (0,1) to calculate 10% threshold
# Changing neuron firing data to 0,1 binary
num_of_cols = len(data_for_freq.columns)
for j in range(12, num_of_cols):  # for all neuron columns
    # setting value to 1 if val>0, 0 otherwise
    data_for_freq[j] = data_for_freq[j].apply(lambda x: 1 if (x > 0) else 0)
    #data_for_freq[j] = data_for_freq[j].apply(binaryIndicator)
"""

trial_indices = []
lap_dictionary = {}
counter=0
for i in range(0, len(unique_trial_ids)):  # for every lap
    idx = int(unique_trial_ids[counter])
    trial_indices.append(idx)
    # associate all lap data with that lap, when there is no odor, 
    # so we can no longer directly associate the dictionary index with the trial number
    lap_dictionary[idx] = data.loc[data[9] == idx]
    counter+=1

print("\n",trial_indices,"\n")

def findShortestTrial():
    min_time = 999999999
    max_time = 0
    shortest_trial = lap_dictionary[1]
    longest_trial = lap_dictionary[1]
    time_list = []
    for trial in lap_dictionary:
        #pair = lap_dictionary[trial].iloc[[0, -1]]
        first = lap_dictionary[trial].iloc[0]
        last = lap_dictionary[trial].iloc[-1]
        time = last[1] - first[1]
        time_list.append(time)
        if(time < min_time):
            min_time=time
            shortest_trial = trial
        if (time > max_time):
            max_time = time
            longest_trial = trial

    #print("AVG trial duration:", sum(time_list)/len(time_list))
    return shortest_trial, min_time, longest_trial, max_time

shortest_trial, min_time,longest_trial, max_time = findShortestTrial()
print("Shortest Trial:", shortest_trial, "Time: ", min_time)
print("Longest Trial:", longest_trial, "Time: ", max_time)


# Determining number of bins
num_of_time_bins = int(min_time // 200)
num_of_space_bins = 4000//100 # 100cm bin width 
print("number of time bins:", num_of_time_bins)


# create empty row to store neuron data frequencies
empty_row = {}
for k in range(0, num_of_cols-13):  # 255 or 459 columns for example
    empty_row[k] = 0.0
def findMeanValues():
    flagCounter = 0
    test_dictionary_for_laps = {}
    for i in lap_dictionary:  # for every trial
        # average columns in that trial to test (eventually, in bin instead)
        current_row = pd.DataFrame(empty_row, index=[0])  # create pandas dataframe from a row of 0's (reset every lap)
        for j in range(12, num_of_cols):  # for all neuron columns
            current_row[j-12] = lap_dictionary[i][j].mean()  # fill each column spot in that row with the mean of all values in that column
            # note: because we set values to (0,1), this mean becomes frequency relative to that trial
        test_dictionary_for_laps[i] = current_row  # set dictionary item equal to that row
    return test_dictionary_for_laps

#test_dictionary_for_laps = findMeanValues()

def timeBinning():
    # each key value pair will be (Trial_number, [ [Bin1], [Bin2], [Bin3], ..., [Bin43] ])
    dict_of_all_bins = {}
    for i in lap_dictionary:  # for every trial
        list_of_bins = {}  # each (key,value) pair will be start-time of bin and list of points in bin for each trial
        # indexed from 1
        for bin_number in range(1, num_of_time_bins+1):
            bin_ending_time = bin_number*200
            last_time = (bin_number-1)*200
            # add all data points from the current trial in lap_dictionary to bin if time is in bin range
            data_points = lap_dictionary[i].loc[(lap_dictionary[i][1] < bin_ending_time) & (lap_dictionary[i][1] >= last_time) ]
            list_of_bins[bin_ending_time] = data_points
        dict_of_all_bins[i] = list_of_bins

    return dict_of_all_bins
time_binned_dict_day5 = timeBinning()
print("temporally binned dictionary")

def spaceBinning():
     # each key value pair will be (Trial_number, [ [Bin1], [Bin2], [Bin3], ..., [Bin43] ])
    dict_of_all_bins = {}
    for i in lap_dictionary:  # for every trial
        list_of_bins = {}  # each (key,value) pair will be start-time of bin and list of points in bin for each trial
        # indexed from 1
        for bin_number in range(1, num_of_space_bins+1):
            bin_starting_dist = bin_number*100  # 10 cm bin width
            last_dist = (bin_number-1)*100
            # add all data points from the current trial in lap_dictionary to bin if time is in bin range
            data_points = lap_dictionary[i].loc[(lap_dictionary[i][7] < bin_starting_dist) & (lap_dictionary[i][7] >= last_dist) ]
            list_of_bins[bin_starting_dist] = data_points
        dict_of_all_bins[i] = list_of_bins

    return dict_of_all_bins
space_binned_dict_day5 = spaceBinning()
print("spatially binned dictionary")

time changed

 [1, 3, 4, 6, 9, 11, 13, 16, 18, 19, 22, 23, 25, 26, 29, 30, 32, 35, 38, 39, 41, 43, 45, 47, 49, 50, 53, 56, 58, 59, 61, 63, 66, 68, 69, 70, 73, 74, 77, 79, 81, 82, 85, 88, 89, 91, 93, 94, 96, 99] 

Shortest Trial: 25 Time:  5701.0
Longest Trial: 32 Time:  19853.0
number of time bins: 28
temporally binned dictionary
spatially binned dictionary


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

## Beginning Shuffle code

In [60]:
print("binned dic")

c = 0
i=0
for trial,bin in binned_dic.items():
    if trial != 0:
        break
    times,cells,spikes = [],[],[]
    for time,activities in bin.items():
        #print("time:", time)
        for cell,v in activities.items():
            c += 1
            if c < 13:
                continue
            if cell == 0:
                continue
            #print("cell:", cell)
            activity_sum = 0
            for x in v:
                #print(x)
                activity_sum += x
            #print("total_activity", activity_sum)
            times.append(time)
            cells.append(cell)
            spikes.append(activity_sum)

binned dic


In [61]:
data_dic = {}
data_dic["times"] = times
data_dic["cells"] = cells
data_dic["spikes"] = spikes

In [62]:
df = pd.DataFrame.from_dict(data_dic)
print(df.head())

   times  cells  spikes
0    200     12     0.0
1    200     13     0.0
2    200     14     0.0
3    200     15     0.0
4    200     16     0.0


In [63]:
df = df.pivot(index='cells', columns='times', values='spikes')
print(df.head())

times  200      400      600      800      1000     1200     1400     1600  \
cells                                                                        
1       NaN  52044.0  52843.0  53643.0  54443.0  55246.0  56044.0  56843.0   
2       NaN      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3       NaN     50.0     56.0     61.0     66.0     73.0     78.0     82.0   
4       NaN      4.0      4.0      4.0      4.0      4.0      6.0     10.0   
5       NaN      4.0      4.0      4.0      4.0      4.0      4.0      4.0   

times     1800     2000  ...     2400     2600     2800     3000     3200  \
cells                    ...                                                
1      57645.0  58443.0  ...  60045.0  60845.0  61645.0  62445.0  63246.0   
2          0.0      0.0  ...      0.0      0.0      0.0      0.0      0.0   
3         86.0     92.0  ...    104.0    104.0    109.0    115.0    122.0   
4         12.0     15.0  ...     20.0     20.0     24.0     27.0    

# Generates Plots of Neuron Activity for each trial

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
import os 
#os.mkdir("./plots")
print("binned dic")

c=0
for trial,bin in binned_dic.items():
    #print("trial:", trial)
    times,cells,spikes = [],[],[]
    for time,activities in bin.items():
        #print("time:", time)
        for cell,v in activities.items():
            if cell<=12:  # to skip over behavior and metadata
                continue
            activity_sum = 0
            count = 0
            for x in v:
                activity_sum += x
                count += 1
            mean_activity = activity_sum / count
            #print("time:", time, "cell:", cell, "mean_act:", mean_activity)
            times.append(time)
            cells.append(cell)
            spikes.append(mean_activity)
    data_dic = {}
    data_dic["times"] = times
    data_dic["cells"] = cells
    data_dic["spikes"] = spikes

    df = pd.DataFrame.from_dict(data_dic)
    df = df.pivot(index='cells', columns='times', values='spikes')
    print(df.head())
    fig, ax = plt.subplots(figsize=(20,20)) 
    plt.title("trial "+str(trial))
    sns.heatmap(df, annot=False)
    plt.savefig("plots/trial"+str(trial)+".png")

binned dic
times  200   400       600   800   1000  1200  1400  1600      1800      2000  \
cells                                                                           
13      0.0   0.0  0.000000   0.0   0.0   0.0   0.0   0.0  0.000000  0.000000   
14      0.0   0.0  0.000000   0.0   0.0   0.0   0.0   0.0  0.000000  0.000000   
15      0.0   0.0  0.000000   0.0   0.0   0.0   0.0   0.0  0.406975  0.720175   
16      0.0   0.0  0.000000   0.0   0.0   0.0   0.0   0.0  0.000000  0.000000   
17      0.0   0.0  0.525425   0.0   0.0   0.0   0.0   0.0  0.000000  0.000000   

times  ...  2400      2600  2800  3000  3200  3400  3600  3800    4000  4200  
cells  ...                                                                    
13     ...   0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  0.0000   0.0  
14     ...   0.0  0.409825   0.0   0.0   0.0   0.0   0.0   0.0  0.0000   0.0  
15     ...   0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  0.0000   0.0  
16     ...   0.0  0.000000

times  200   400   600   800   1000  1200  1400      1600  1800  2000  ...  \
cells                                                                  ...   
13      0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   
14      0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   
15      0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.954875   0.0   0.0  ...   
16      0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   
17      0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   

times  2400  2600  2800  3000  3200  3400  3600  3800  4000  4200  
cells                                                              
13      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
14      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
15      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
16      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
17      0.0   0.0   0.0   0.0   0.0   0.0   0

times  200   400   600   800   1000  1200  1400  1600  1800  2000  ...  2400  \
cells                                                              ...         
13      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
14      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
15      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
16      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
17      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

times  2600  2800  3000  3200  3400  3600  3800  4000  4200  
cells                                                        
13      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
14      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
15      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
16      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
17      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 r

/home/sophia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


times  200   400   600   800   1000  1200  1400  1600  1800  2000  ...  2400  \
cells                                                              ...         
13      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
14      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
15      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
16      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
17      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

times  2600  2800  3000  3200  3400  3600  3800  4000  4200  
cells                                                        
13      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
14      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
15      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
16      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
17      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 r

times  200   400   600   800   1000  1200  1400  1600    1800  2000  ...  \
cells                                                                ...   
13      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0000   0.0  ...   
14      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0000   0.0  ...   
15      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  1.6963   0.0  ...   
16      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0000   0.0  ...   
17      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0000   0.0  ...   

times  2400  2600  2800  3000  3200  3400  3600  3800  4000  4200  
cells                                                              
13      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
14      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
15      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
16      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
17      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0

# Re-shapes data for shuffle test

In [64]:
import matplotlib.pyplot as plt
import seaborn as sns
import os 

times,cells,spikes,t = [],[],[],[]
data_dic = {}
for trial,bin in binned_dic.items():
    for time,activities in bin.items():
        #print("time:", time)
        for cell,v in activities.items():
            if cell <= 12:  # to skip over behavior and metadata
                continue
            activity_sum = 0
            count = 0
            for x in v:
                activity_sum += x
                count += 1
            mean_activity = activity_sum / count
            t.append(trial)
            times.append(time)
            cells.append(cell)
            spikes.append(mean_activity)
    #data_dic["trial"] = trials
    data_dic["trial"] = t
    data_dic["times"] = times
    data_dic["cells"] = cells
    data_dic["spikes"] = spikes

df = pd.DataFrame.from_dict(data_dic)

print(df.head())

   trial  times  cells  spikes
0      0    200     13     0.0
1      0    200     14     0.0
2      0    200     15     0.0
3      0    200     16     0.0
4      0    200     17     0.0


# Does shuffle test for day 0, pre-reward

In [65]:
def shuffle_cells(df):
    df1 = pd.DataFrame(columns=['trials','times','cells','spikes'])
    for i in range(1,256):
        cell_i = df[df["cells"] == i]
        cell_i["spikes"] = np.random.permutation(cell_i["spikes"])
        df1 = df1.append(cell_i, ignore_index=True)
    return df1

        
max_spike = {}
shuffled_spike = {}
for i in range(0,100):
    df1 = shuffle_cells(df)
    for i, j in df1.iterrows():
        time = int(j["times"])
        cell = int(j["cells"])
        spike = float(j["spikes"])
        if (time, cell) in shuffled_spike:
            shuffled_spike[(time,cell)].append(spike)
        else:
            shuffled_spike[(time,cell)] = []
            shuffled_spike[(time,cell)].append(spike)       
    for k,v in shuffled_spike.items():
        max_spike[k] = max(v)
            
print(shuffled_spike)  
print(max_spike)

/home/sophia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [71]:
from scipy import stats

time_cells = {}
for i, j in df.iterrows():
        time = int(j["times"])
        cell = int(j["cells"])
        spike = float(j["spikes"])
        trial = int(j["trial"])
        if (time,cell) in shuffled_spike:
            arr = np.array(shuffled_spike[(time,cell)])
        else:
            continue
        arr = np.append(arr,spike)
        std_dev = np.std(arr)
        z_scores = stats.zscore(arr)
        if np.all(arr==0):
            continue
        res = dict(zip(arr, z_scores)) 
        nintey_fifth_p = np.percentile(arr, 95)
        if nintey_fifth_p == 0:
            continue
        if spike > nintey_fifth_p:
            z_score = res[spike]
            #if spike > z_score + std_dev:
            if True:
                if cell in time_cells:
                    time_cells[(time, cell)] += 1
                else:
                    time_cells[(time,cell)] = 1
                    
print(time_cells)

{(200, 139): 1, (400, 137): 1, (600, 88): 1, (600, 120): 1, (600, 137): 1, (800, 120): 1, (1000, 77): 1, (1000, 137): 1, (1200, 65): 1, (1200, 77): 1, (1200, 137): 1, (1200, 139): 1, (1400, 77): 1, (1400, 139): 1, (1600, 30): 1, (1600, 31): 1, (1600, 207): 1, (1800, 30): 1, (1800, 31): 1, (1800, 88): 1, (1800, 137): 1, (1800, 189): 1, (2000, 18): 1, (2000, 189): 1, (2200, 18): 1, (2200, 96): 1, (2400, 88): 1, (2400, 96): 1, (3200, 45): 1, (3200, 96): 1, (3600, 148): 1, (3800, 31): 1, (3800, 88): 1, (3800, 96): 1, (4000, 43): 1, (400, 96): 1, (600, 30): 1, (600, 148): 1, (1000, 43): 1, (1200, 96): 1, (2200, 189): 1, (2200, 207): 1, (2400, 30): 1, (2400, 139): 1, (2600, 45): 1, (2600, 88): 1, (2600, 120): 1, (2600, 139): 1, (2800, 31): 1, (2800, 139): 1, (3000, 30): 1, (3200, 137): 1, (3400, 30): 1, (3800, 18): 1, (4000, 137): 1, (4200, 88): 1, (200, 43): 1, (200, 124): 1, (800, 139): 1, (1000, 120): 1, (1200, 20): 1, (1200, 120): 1, (1200, 124): 1, (1400, 124): 1, (1600, 96): 1, (2000, 

In [72]:
listypoo = []
for k,v in time_cells.items():
    #print(k, v)
    listypoo.append(k[1])
    
print(set(listypoo))

{137, 139, 145, 18, 148, 20, 151, 27, 30, 31, 158, 35, 43, 45, 189, 63, 65, 202, 77, 207, 82, 88, 96, 120, 124}


In [20]:
time_cells = {}
for i, j in df.iterrows():
    time = int(j["times"])
    cell = int(j["cells"])
    spike = int(j["spikes"])
    trial = int(j["trial"])
    average = avrg_spike[(time,cell)]
    if spike > (3*average):
        if cell in time_cells:
            time_cells[cell] += 1
        else:
            time_cells[cell] = 1
print(time_cells)

trial: 0 cell: 7 bin: 200
trial: 0 cell: 11 bin: 200
trial: 0 cell: 34 bin: 200
trial: 0 cell: 168 bin: 200
trial: 0 cell: 170 bin: 200
trial: 0 cell: 220 bin: 200
trial: 0 cell: 7 bin: 400
trial: 0 cell: 11 bin: 400
trial: 0 cell: 54 bin: 400
trial: 0 cell: 7 bin: 600
trial: 0 cell: 11 bin: 600
trial: 0 cell: 54 bin: 600
trial: 0 cell: 156 bin: 600
trial: 0 cell: 184 bin: 600
trial: 0 cell: 206 bin: 600
trial: 0 cell: 7 bin: 800
trial: 0 cell: 11 bin: 800
trial: 0 cell: 24 bin: 800
trial: 0 cell: 58 bin: 800
trial: 0 cell: 68 bin: 800
trial: 0 cell: 70 bin: 800
trial: 0 cell: 98 bin: 800
trial: 0 cell: 100 bin: 800
trial: 0 cell: 156 bin: 800
trial: 0 cell: 170 bin: 800
trial: 0 cell: 263 bin: 800
trial: 0 cell: 7 bin: 1000
trial: 0 cell: 24 bin: 1000
trial: 0 cell: 51 bin: 1000
trial: 0 cell: 52 bin: 1000
trial: 0 cell: 68 bin: 1000
trial: 0 cell: 213 bin: 1000
trial: 0 cell: 214 bin: 1000
trial: 0 cell: 227 bin: 1000
trial: 0 cell: 236 bin: 1000
trial: 0 cell: 7 bin: 1200
trial: 0 c

trial: 1 cell: 214 bin: 8000
trial: 1 cell: 236 bin: 8200
trial: 1 cell: 74 bin: 8400
trial: 1 cell: 77 bin: 8600
trial: 2 cell: 70 bin: 200
trial: 2 cell: 76 bin: 200
trial: 2 cell: 82 bin: 200
trial: 2 cell: 95 bin: 200
trial: 2 cell: 142 bin: 200
trial: 2 cell: 145 bin: 200
trial: 2 cell: 214 bin: 200
trial: 2 cell: 253 bin: 200
trial: 2 cell: 66 bin: 400
trial: 2 cell: 95 bin: 400
trial: 2 cell: 132 bin: 400
trial: 2 cell: 152 bin: 400
trial: 2 cell: 176 bin: 400
trial: 2 cell: 234 bin: 400
trial: 2 cell: 41 bin: 800
trial: 2 cell: 189 bin: 800
trial: 2 cell: 41 bin: 1000
trial: 2 cell: 89 bin: 1000
trial: 2 cell: 134 bin: 1000
trial: 2 cell: 189 bin: 1000
trial: 2 cell: 93 bin: 1200
trial: 2 cell: 134 bin: 1200
trial: 2 cell: 161 bin: 1200
trial: 2 cell: 193 bin: 1200
trial: 2 cell: 219 bin: 1200
trial: 2 cell: 54 bin: 1400
trial: 2 cell: 73 bin: 1400
trial: 2 cell: 74 bin: 1400
trial: 2 cell: 98 bin: 1400
trial: 2 cell: 144 bin: 1400
trial: 2 cell: 150 bin: 1400
trial: 2 cell: 22

trial: 3 cell: 134 bin: 4600
trial: 3 cell: 168 bin: 4600
trial: 3 cell: 177 bin: 4600
trial: 3 cell: 180 bin: 4600
trial: 3 cell: 102 bin: 4800
trial: 3 cell: 171 bin: 4800
trial: 3 cell: 20 bin: 5000
trial: 3 cell: 66 bin: 5000
trial: 3 cell: 118 bin: 5000
trial: 3 cell: 171 bin: 5000
trial: 3 cell: 88 bin: 5200
trial: 3 cell: 121 bin: 5200
trial: 3 cell: 255 bin: 5200
trial: 3 cell: 88 bin: 5400
trial: 3 cell: 141 bin: 5400
trial: 3 cell: 159 bin: 5400
trial: 3 cell: 226 bin: 5400
trial: 3 cell: 232 bin: 5400
trial: 3 cell: 35 bin: 5600
trial: 3 cell: 42 bin: 5600
trial: 3 cell: 66 bin: 5600
trial: 3 cell: 87 bin: 5600
trial: 3 cell: 222 bin: 5600
trial: 3 cell: 247 bin: 5600
trial: 3 cell: 34 bin: 5800
trial: 3 cell: 35 bin: 5800
trial: 3 cell: 106 bin: 5800
trial: 3 cell: 222 bin: 5800
trial: 3 cell: 255 bin: 5800
trial: 3 cell: 56 bin: 6000
trial: 3 cell: 125 bin: 6000
trial: 3 cell: 154 bin: 6000
trial: 3 cell: 190 bin: 6000
trial: 3 cell: 192 bin: 6000
trial: 3 cell: 206 bin: 6

trial: 5 cell: 97 bin: 4000
trial: 5 cell: 178 bin: 4000
trial: 5 cell: 205 bin: 4000
trial: 5 cell: 243 bin: 4000
trial: 5 cell: 244 bin: 4000
trial: 5 cell: 58 bin: 4200
trial: 5 cell: 97 bin: 4200
trial: 5 cell: 167 bin: 4200
trial: 5 cell: 41 bin: 4400
trial: 5 cell: 105 bin: 4400
trial: 5 cell: 57 bin: 4600
trial: 5 cell: 134 bin: 4600
trial: 5 cell: 18 bin: 4800
trial: 5 cell: 90 bin: 4800
trial: 5 cell: 210 bin: 4800
trial: 5 cell: 28 bin: 5000
trial: 5 cell: 90 bin: 5000
trial: 5 cell: 92 bin: 5000
trial: 5 cell: 171 bin: 5200
trial: 5 cell: 177 bin: 5200
trial: 5 cell: 102 bin: 5400
trial: 5 cell: 168 bin: 5400
trial: 5 cell: 180 bin: 5400
trial: 5 cell: 234 bin: 5400
trial: 5 cell: 66 bin: 5600
trial: 5 cell: 96 bin: 5600
trial: 5 cell: 113 bin: 5600
trial: 5 cell: 168 bin: 5600
trial: 5 cell: 204 bin: 5600
trial: 5 cell: 43 bin: 5800
trial: 5 cell: 68 bin: 5800
trial: 5 cell: 113 bin: 5800
trial: 5 cell: 225 bin: 5800
trial: 5 cell: 54 bin: 6000
trial: 5 cell: 89 bin: 6000
t

trial: 7 cell: 131 bin: 600
trial: 7 cell: 137 bin: 600
trial: 7 cell: 142 bin: 600
trial: 7 cell: 26 bin: 800
trial: 7 cell: 73 bin: 800
trial: 7 cell: 76 bin: 800
trial: 7 cell: 95 bin: 800
trial: 7 cell: 132 bin: 800
trial: 7 cell: 222 bin: 800
trial: 7 cell: 214 bin: 1000
trial: 7 cell: 219 bin: 1000
trial: 7 cell: 224 bin: 1400
trial: 7 cell: 261 bin: 1400
trial: 7 cell: 74 bin: 1600
trial: 7 cell: 98 bin: 1600
trial: 7 cell: 152 bin: 1600
trial: 7 cell: 159 bin: 1600
trial: 7 cell: 189 bin: 1600
trial: 7 cell: 70 bin: 1800
trial: 7 cell: 86 bin: 1800
trial: 7 cell: 94 bin: 1800
trial: 7 cell: 161 bin: 1800
trial: 7 cell: 162 bin: 1800
trial: 7 cell: 189 bin: 1800
trial: 7 cell: 262 bin: 1800
trial: 7 cell: 93 bin: 2000
trial: 7 cell: 189 bin: 2000
trial: 7 cell: 73 bin: 2200
trial: 7 cell: 86 bin: 2200
trial: 7 cell: 134 bin: 2200
trial: 7 cell: 51 bin: 2400
trial: 7 cell: 58 bin: 2400
trial: 7 cell: 222 bin: 2600
trial: 7 cell: 159 bin: 2800
trial: 7 cell: 107 bin: 3000
trial: 7

...

trial: 20 cell: 211 bin: 8200
trial: 20 cell: 218 bin: 8200
trial: 20 cell: 89 bin: 8400
trial: 20 cell: 114 bin: 8400
trial: 20 cell: 114 bin: 8600
trial: 20 cell: 186 bin: 8600
trial: 20 cell: 218 bin: 8600
trial: 21 cell: 130 bin: 200
trial: 21 cell: 216 bin: 400
trial: 21 cell: 231 bin: 400
trial: 21 cell: 134 bin: 800
trial: 21 cell: 86 bin: 1000
trial: 21 cell: 109 bin: 1000
trial: 21 cell: 134 bin: 1000
trial: 21 cell: 86 bin: 1200
trial: 21 cell: 135 bin: 1200
trial: 21 cell: 189 bin: 1200
trial: 21 cell: 213 bin: 1200
trial: 21 cell: 64 bin: 1400
trial: 21 cell: 93 bin: 1400
trial: 21 cell: 208 bin: 1600
trial: 21 cell: 28 bin: 1800
trial: 21 cell: 102 bin: 1800
trial: 21 cell: 250 bin: 1800
trial: 21 cell: 41 bin: 2000
trial: 21 cell: 64 bin: 2000
trial: 21 cell: 224 bin: 2000
trial: 21 cell: 239 bin: 2000
trial: 21 cell: 246 bin: 2000
trial: 21 cell: 49 bin: 2200
trial: 21 cell: 64 bin: 2200
trial: 21 cell: 107 bin: 2200
trial: 21 cell: 128 bin: 2200
trial: 21 cell: 184 bin:

trial: 23 cell: 142 bin: 1200
trial: 23 cell: 214 bin: 1200
trial: 23 cell: 76 bin: 1400
trial: 23 cell: 132 bin: 1400
trial: 23 cell: 149 bin: 1400
trial: 23 cell: 158 bin: 1400
trial: 23 cell: 210 bin: 1400
trial: 23 cell: 214 bin: 1400
trial: 23 cell: 154 bin: 1800
trial: 23 cell: 214 bin: 1800
trial: 23 cell: 180 bin: 2000
trial: 23 cell: 205 bin: 2000
trial: 23 cell: 37 bin: 2200
trial: 23 cell: 161 bin: 2200
trial: 23 cell: 189 bin: 2200
trial: 23 cell: 205 bin: 2200
trial: 23 cell: 88 bin: 2400
trial: 23 cell: 161 bin: 2400
trial: 23 cell: 189 bin: 2400
trial: 23 cell: 73 bin: 2600
trial: 23 cell: 86 bin: 2600
trial: 23 cell: 98 bin: 2600
trial: 23 cell: 238 bin: 2600
trial: 23 cell: 213 bin: 3000
trial: 23 cell: 152 bin: 3200
trial: 23 cell: 86 bin: 3400
trial: 23 cell: 134 bin: 3400
trial: 23 cell: 145 bin: 3400
trial: 23 cell: 38 bin: 3600
trial: 23 cell: 58 bin: 3600
trial: 23 cell: 134 bin: 3600
trial: 23 cell: 185 bin: 3600
trial: 23 cell: 208 bin: 3600
trial: 23 cell: 58 

trial: 25 cell: 76 bin: 800
trial: 25 cell: 132 bin: 800
trial: 25 cell: 98 bin: 1000
trial: 25 cell: 145 bin: 1000
trial: 25 cell: 152 bin: 1000
trial: 25 cell: 214 bin: 1000
trial: 25 cell: 86 bin: 1400
trial: 25 cell: 98 bin: 1400
trial: 25 cell: 233 bin: 1400
trial: 25 cell: 94 bin: 1600
trial: 25 cell: 135 bin: 1600
trial: 25 cell: 175 bin: 1600
trial: 25 cell: 185 bin: 1600
trial: 25 cell: 98 bin: 1800
trial: 25 cell: 185 bin: 1800
trial: 25 cell: 238 bin: 1800
trial: 25 cell: 250 bin: 1800
trial: 25 cell: 28 bin: 2200
trial: 25 cell: 64 bin: 2200
trial: 25 cell: 89 bin: 2200
trial: 25 cell: 107 bin: 2200
trial: 25 cell: 128 bin: 2200
trial: 25 cell: 150 bin: 2200
trial: 25 cell: 213 bin: 2200
trial: 25 cell: 224 bin: 2200
trial: 25 cell: 246 bin: 2200
trial: 25 cell: 21 bin: 2400
trial: 25 cell: 27 bin: 2400
trial: 25 cell: 45 bin: 2400
trial: 25 cell: 64 bin: 2400
trial: 25 cell: 87 bin: 2400
trial: 25 cell: 102 bin: 2400
trial: 25 cell: 45 bin: 2600
trial: 25 cell: 64 bin: 260

trial: 27 cell: 120 bin: 400
trial: 27 cell: 76 bin: 800
trial: 27 cell: 109 bin: 800
trial: 27 cell: 132 bin: 800
trial: 27 cell: 143 bin: 800
trial: 27 cell: 152 bin: 800
trial: 27 cell: 214 bin: 800
trial: 27 cell: 229 bin: 800
trial: 27 cell: 98 bin: 1000
trial: 27 cell: 98 bin: 1200
trial: 27 cell: 145 bin: 1200
trial: 27 cell: 86 bin: 1400
trial: 27 cell: 52 bin: 1600
trial: 27 cell: 175 bin: 1600
trial: 27 cell: 185 bin: 1600
trial: 27 cell: 213 bin: 1800
trial: 27 cell: 246 bin: 1800
trial: 27 cell: 45 bin: 2000
trial: 27 cell: 50 bin: 2000
trial: 27 cell: 122 bin: 2000
trial: 27 cell: 45 bin: 2200
trial: 27 cell: 49 bin: 2200
trial: 27 cell: 89 bin: 2200
trial: 27 cell: 136 bin: 2400
trial: 27 cell: 150 bin: 2400
trial: 27 cell: 208 bin: 2400
trial: 27 cell: 42 bin: 2600
trial: 27 cell: 69 bin: 2600
trial: 27 cell: 87 bin: 2600
trial: 27 cell: 89 bin: 2600
trial: 27 cell: 91 bin: 2600
trial: 27 cell: 41 bin: 2800
trial: 27 cell: 69 bin: 2800
trial: 27 cell: 112 bin: 2800
trial

trial: 29 cell: 89 bin: 2200
trial: 29 cell: 150 bin: 2200
trial: 29 cell: 38 bin: 2400
trial: 29 cell: 41 bin: 2400
trial: 29 cell: 69 bin: 2400
trial: 29 cell: 128 bin: 2400
trial: 29 cell: 182 bin: 2400
trial: 29 cell: 38 bin: 2600
trial: 29 cell: 42 bin: 2600
trial: 29 cell: 69 bin: 2600
trial: 29 cell: 107 bin: 2600
trial: 29 cell: 128 bin: 2600
trial: 29 cell: 42 bin: 2800
trial: 29 cell: 91 bin: 2800
trial: 29 cell: 150 bin: 2800
trial: 29 cell: 230 bin: 2800
trial: 29 cell: 243 bin: 2800
trial: 29 cell: 92 bin: 3000
trial: 29 cell: 124 bin: 3000
trial: 29 cell: 153 bin: 3000
trial: 29 cell: 210 bin: 3000
trial: 29 cell: 83 bin: 3200
trial: 29 cell: 90 bin: 3200
trial: 29 cell: 97 bin: 3200
trial: 29 cell: 68 bin: 3400
trial: 29 cell: 113 bin: 3600
trial: 29 cell: 136 bin: 3600
trial: 29 cell: 247 bin: 3600
trial: 29 cell: 19 bin: 3800
trial: 29 cell: 59 bin: 3800
trial: 29 cell: 66 bin: 3800
trial: 29 cell: 196 bin: 3800
trial: 29 cell: 213 bin: 3800
trial: 29 cell: 219 bin: 38